# NLP word embedding

This include all NLP approaches for word embedding

Prediction of Likes and Retweets Using Text Information Retrieval
https://ai.intelligentonlinetools.com/ml/text-clustering-doc2vec-word-embedding-machine-learning/

Paper:
1-s2.0-S1877050920304129-main.pdf


Le github magique:
https://github.com/buomsoo-kim/Word-embedding-with-Python

French corpus: https://stackoverflow.com/questions/42058396/python-nltk-and-textblob-in-french

## Word embedding à la main avec Doc 2 Vec

In [ ]:
import re

from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.corpus import gutenberg
from multiprocessing import Pool
from scipy import spatial




In [1]:
#télcharge "quinze essais politiques" dans data et en fait une liste
# opening the file in read mode
my_file = open("data/quinze_essais_politiques.txt", "r")
  
# reading the file
corpus = my_file.read()
#remove \n
corpus = corpus.replace('\n', ' ')
corpus = corpus.replace('.', ',')

  
# replacing end splitting the text 
# when newline ('\n') is seen.
corpus = corpus.split(",")
my_file.close()
corpus[5]

'              David Hume “Essai sur la liberté de la presse”'

Use re module to preprocess data

Convert all letters into lowercase

Remove punctuations, numbers, etc

In [ ]:
for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
    #remove punctuation
    corpus[i] = re.sub(r'[^\w\s]','',corpus[i])
    #make a list of corpus[i]
    corpus[i] = corpus[i].split()
    

corpus[5]


For the doc2vec model, input data should be in format of iterable TaggedDocuments"

Each TaggedDocument instance comprises words and tags

Hence, each document (i.e., a sentence or paragraph) should have a unique tag which is identifiable

In [2]:
for i in range(len(corpus)):
    corpus[i] = TaggedDocument(words = corpus[i], tags = ['sent{}'.format(i)])    # converting each sentence into a TaggedDocument
corpus[5]

NameError: name 'TaggedDocument' is not defined

In [3]:
model = Doc2Vec(documents = corpus, vector_size = 100, min_count = 1)
model.init_sims(replace = True)

model.save('doc2vec_model')
model = Doc2Vec.load('doc2vec_model')


NameError: name 'Doc2Vec' is not defined

In [ ]:
v1 = model.infer_vector(['Macron',' démission'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
v2 = model.infer_vector(['gilets jaune'])    # in doc2vec, infer_vector() function is used to infer the vector embedding of a document
# define a function that computes cosine similarity between two words
def cosine_similarity(v1, v2):
    return 1 - spatial.distance.cosine(v1, v2)
cosine_similarity(v1, v2)


# Word embedding with transfer learning

In [ ]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('fr_core_news_sm')
# process a sentence using the model
doc = nlp("Macron démission")
# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc.vector


# Model with Spacy text embedding



## Data

In [ ]:
# Load the training data
train_data = pd.read_csv("data/train.csv")

# Load the evaluation data
eval_data = pd.read_csv("data/evaluation.csv")


# Here we split our training data into trainig and testing set. This way we can estimate the evaluation of our model without uploading to Kaggle and avoid overfitting over our evaluation dataset.
# scsplit method is used in order to split our regression data in a stratisfied way and keep a similar distribution of retweet counts between the two sets
X_train, X_test, y_train, y_test = scsplit(train_data, train_data['retweets_count'], stratify=train_data['retweets_count'], train_size=0.7, test_size=0.3)
# We remove the actual number of retweets from our features since it is the value that we are trying to predict
X_train = X_train.drop(['retweets_count'], axis=1)
X_test = X_test.drop(['retweets_count'], axis=1)

In [ ]:
#use nlp model fro text train data
X_train_txt = X_train['text'].apply(lambda x: nlp(x).vector)
#use nlp model fro text eval data
X_test_txt = X_test['text'].apply(lambda x: nlp(x).vector)
# We fit our model using the training data
reg = RamdomForestRegressor()
reg.fit(X_train_test, y_train)
# Predict the number of retweets for the evaluation dataset
y_pred = reg.predict(X_test_txt)
# We want to make sure that all predictions are non-negative integers
y_pred = [int(value) if value >= 0 else 0 for value in y_pred]

print("Text RF", mean_absolute_error(y_true=y_test, y_pred=y_pred))